<a href="https://colab.research.google.com/github/Katanov/DA_projects/blob/main/TestovoeSQL_OK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
#!gdown --id 1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
#!mv _sqlite3.cpython-37m-x86_64-linux-gnu.so /usr/lib/python3.7/lib-dynload/
#import os
#os.kill(os.getpid(), 9)

In [35]:
import pandas as pd
import numpy as np

In [36]:
import sqlite3

In [37]:
sqlite3.sqlite_version

'3.36.0'

In [38]:
con = sqlite3.connect('db')

#### Обычная SQL задача 1

In [39]:
df = pd.read_csv('/content/Новая таблица - Departments.csv')
df1 = pd.read_csv('/content/Новая таблица - Employees.csv')

In [40]:
df.to_sql('Dep',con,index=False,if_exists='replace')
df1.to_sql('Emp',con,index=False,if_exists='replace')

In [41]:
def select(sql):
  return pd.read_sql(sql,con)

- a) Для каждого сотрудника найти его департамент, включая тех, у кого
департамента нет

In [62]:
sql = '''
select emp.name, dep.name as dep_name from emp
left join dep on emp.dep_id = dep.id
'''

In [63]:
select(sql)

,name,dep_name
0,John Smith,Finance
1,Jack Smith,None
2,Becky Smith,Finance
3,Rebecca Smith,Operations
4,Sonny Smith,Deployment


Данных мало, но можно предположить, что определить отдел можно по manager_id. У первого и второго отдела - всегда второй менеджер. А у третьего отдела - первый. В таком случае решение будет выглядть чуть-чуть иначе:

In [44]:
sql = '''
with real_deal as
(select distinct dep_id, Manager_id from emp
where dep_id not null and Manager_id not null ),

fin as 
(select *,
COALESCE (Dep_id, (select dep_id from real_deal
where real_deal.manager_id = emp.manager_id
)) as department
from emp)

select fin.name, dep.name from fin
left join dep on fin.department = dep.id
'''

In [45]:
select(sql)

,name,name
0,John Smith,Finance
1,Jack Smith,Deployment
2,Becky Smith,Finance
3,Rebecca Smith,Operations
4,Sonny Smith,Deployment


- b) Найти наибольшую зарплату по департаментам и отсортировать
департаменты по убыванию максимальной зарплаты

In [46]:
sql = '''
select dep.name, max(salary) as max_sal from emp
left join dep on emp.dep_id = dep.id
group by dep.name
order by max_sal desc
'''

In [47]:
select(sql)

,name,max_sal
0,Deployment,3000
1,Finance,2000
2,None,1500
3,Operations,700


#### Обычная SQL задача 2

In [48]:
df2 = pd.read_csv('/content/Новая таблица - Purchases.csv')

In [49]:
df2.to_sql('pur',con,index=False,if_exists='replace')

- a) Посчитай доход с женской аудитории (доход= сумма price*items)

In [50]:
sql = '''
select sum(items * price) as rev from pur
where user_gender like 'f%'
'''

In [51]:
select (sql)

,rev
0,500


- b) Сравни доход по группе мужчин и женщин

In [52]:
sql = '''
select case when user_gender like 'f%' then "female" else "male" end as real_gender,
sum(items * price) as rev from pur
group by real_gender
'''

In [53]:
select (sql)

,real_gender,rev
0,female,500
1,male,200


- c) Посчитай кол-во уникальных пользователей-мужчин, заказавших более чем три айтема (суммарно за все заказы).

In [54]:
sql = '''
select count (distinct user_id) as cnt,
case when user_gender like 'f%' then "female" else "male" end as real_gender
-- sum(items) as sum_items
from pur
where real_gender == "male"
group by real_gender
having sum(items) >= 2
'''

In [55]:
select (sql)

,cnt,real_gender
0,2,male


В запросе указал больше или равно двух айтемов (вместо трех по условию), чтобы запрос выдал хоть какой-то результат :)

#### Не самая обычная SQL задача 3

Каждый пользователь имеет набор транзакций с определенным
временем. 
- Используя таблицы выше, напиши НАИБОЛЕЕ ОПТИМАЛЬНЫЕ
SQL запросы:

In [56]:
df3 = pd.read_csv('/content/Новая таблица - Transactions.csv')

In [57]:
df3.to_sql('tran',con,index=False,if_exists='replace')

- a) Выведи для каждого пользователя первое наименование, которое он
заказал (первое по времени транзакции)

In [58]:
sql = '''
select user_id, item from tran
group by user_id
order by min(transaction_ts)
'''

In [59]:
select(sql)

,user_id,item
0,3378024101,dress
1,13811335,glove


- b) Посчитай сколько транзакций в среднем делает каждый пользователь
в течении 72х часов с момента первой транзакции

In [60]:
sql = '''
select avg(cnt) from
(select count(*) as cnt, t.user_id
from tran t
where t.transaction_ts > (select min(transaction_ts) from tran where t.user_id = user_id)
and t.transaction_ts < (select datetime(min(transaction_ts), '+72 hours') from tran where t.user_id = user_id)
group by t.user_id)
'''

In [61]:
select(sql)

,avg(cnt)
0,3.5
